In [1]:
pip install contractions

In [2]:
import numpy as np
import pandas as pd
import spacy
import re
import nltk
import contractions
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification
from transformers import pipeline


In [4]:
!pip install openpyxl

In [5]:
test=pd.read_excel('../input/enterpret-absa/test.xlsx')


In [6]:
nlp = spacy.load('en_core_web_lg')

In [8]:
def preprocess(text):
    #to lowercase
    text = str(text).lower()
    #remove urls
   
    #remove emails
    text = re.sub(r'\S*@\S*\s?',' ',text)
    #remove mentions
    text = re.sub(r'@\S+', ' ', text)
    #contractions
    text = contractions.fix(text)
    #remove hashtags
    text = re.sub(r'@\S+', ' ', text)
    #remove emojis
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    #remove all punct
    text = re.sub('[^A-z0-9]', ' ', text)
    #remove extras whitespaces
    text = re.sub(' +', ' ', text)
    return text

In [9]:

qa_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_model = pipeline('question-answering', model=qa_model, tokenizer=qa_tokenizer)

In [10]:

sent_tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
sent_model = pipeline('sentiment-analysis',model="finiteautomata/bertweet-base-sentiment-analysis",tokenizer=sent_tokenizer)

In [11]:
def get_sentiment(aspect,text,qa_model,sent_model):
    
    question = f'how is {aspect}'
    QA_input = {'question': question, 'context': text}
    qa_result = qa_model(QA_input)
    answer = qa_result['answer']
    print(question)
    print(answer)
      #sentiment model 
    sent_result = sent_model(answer)    
    print(sent_result)
    sentiment = sent_result[0]['label']
    print(sentiment)

    if sentiment == 'NEG':
        sentiment, score = 'Negative', 0
    elif sentiment == 'NEU':
        sentiment, score = 'Neutral', 1
    else:
        sentiment, score = 'Positive', 2
    

    return sentiment

In [16]:
def compute(text, aspect, qa_model, sent_model):
    text = preprocess(text)
    #noun_list = get_noun(preprocess_text)
    #aspect_classes = get_similar_words(noun_list, aspects)
   # print(aspect_classes)
    #print(noun_list)
    aspect=preprocess(aspect)
    sentiment_result = get_sentiment(aspect, text, qa_model, sent_model)
    return sentiment_result
    
   
    

In [20]:
text = input("Enter your text/context: ")
print(text)

In [14]:
aspect= input("Enter aspect ")
print(aspect)